# 02 Baseline

In [1]:
import config
from utils import data as utils_data
from utils.model import Model

import pandas as pd
import numpy as np
import tensorflow as tf

## Load data

In [2]:
data, imgs_left, imgs_right = utils_data.load(
    config.PATH_DATA, config.PATH_IMGS_LEFT, config.PATH_IMGS_RIGHT
)

In [3]:
data.head()

,eye_right_x,eye_right_y,eye_right_width,eye_right_image,face_width,eye_left_width,face_height,eye_left_x,eye_left_y,eye_right_height,face_x,face_y,eye_left_height,y,x,eye_left_image,glasses
0,0.051562,0.456944,0.171875,1c322a8bbd645b20da263dd342dc20ce_eye_right_0.jpg,0.503125,0.171875,0.447222,-0.162500,0.454167,0.081944,-0.232813,0.384722,0.077778,0.585938,-0.998536,1c322a8bbd645b20da263dd342dc20ce_eye_left_0.jpg,False
1,0.050000,0.459722,0.175000,1c322a8bbd645b20da263dd342dc20ce_eye_right_1.jpg,0.503125,0.165625,0.447222,-0.160937,0.456944,0.070833,-0.232813,0.384722,0.059722,0.585938,-0.998536,1c322a8bbd645b20da263dd342dc20ce_eye_left_1.jpg,False
2,0.050000,0.458333,0.171875,1c322a8bbd645b20da263dd342dc20ce_eye_right_2.jpg,0.503125,0.171875,0.447222,-0.160937,0.454167,0.077778,-0.232813,0.384722,0.075000,0.585938,-0.998536,1c322a8bbd645b20da263dd342dc20ce_eye_left_2.jpg,False
3,0.054688,0.461111,0.168750,1c322a8bbd645b20da263dd342dc20ce_eye_right_3.jpg,0.503125,0.162500,0.447222,-0.157813,0.455556,0.070833,-0.232813,0.384722,0.066667,0.585938,-0.998536,1c322a8bbd645b20da263dd342dc20ce_eye_left_3.jpg,False
4,0.054688,0.456944,0.165625,1c322a8bbd645b20da263dd342dc20ce_eye_right_4.jpg,0.503125,0.162500,0.447222,-0.157813,0.455556,0.075000,-0.232813,0.384722,0.066667,0.585938,-0.998536,1c322a8bbd645b20da263dd342dc20ce_eye_left_4.jpg,False


## Split data

In [4]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = utils_data.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [5]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

Train length: 43647
Validation length: 2298
Test length: 2419


## Model

### Architecture

In [6]:
def get_model(features, left_imgs, right_imgs, *args, **kwargs):
    with tf.variable_scope('model'):
        left_flat = tf.contrib.layers.flatten (left_imgs)
        right_flat =  tf.contrib.layers.flatten (right_imgs)
        flat_input = tf.concat(
            values=[features, left_flat, right_flat],
            axis=1,
            name='concat'
        )
        # 12+(20x30)x2 = 1212 inputs
        d1 = tf.layers.dense(flat_input, 1024, activation=tf.nn.relu)
        d2 = tf.layers.dense(d1, 512, activation=tf.nn.relu)
        d3 = tf.layers.dense(d2, 128, activation=tf.nn.relu)
        d4 = tf.layers.dense(d3, 2, activation=None)
    return d4

### Parameters

In [7]:
MODEL_NAME = 'baseline-05'

EPOCHS = 1
BATCH_SIZE = 128
LEARNING_RATE = 0.001

## Runs

In [8]:
model = Model(MODEL_NAME, get_model)

### Train

In [9]:
model.train(
    train_data, train_imgs_left, train_imgs_right,
    validation_data, validation_imgs_left, validation_imgs_right,
    BATCH_SIZE, EPOCHS, LEARNING_RATE
)

Epoch: 1 of 1
	Train VS Validation: 0.5744094848632812 0.5727275013923645
	Train VS Validation: 0.5422289371490479 0.4859340190887451
	Train VS Validation: 0.41297852993011475 0.44618117809295654
	Train VS Validation: 0.3883928656578064 0.3999041020870209
	Train VS Validation: 0.3913787603378296 0.3899846374988556
	Train VS Validation: 0.3714675307273865 0.38523632287979126
	Train VS Validation: 0.4630874693393707 0.40230482816696167
	Train VS Validation: 0.37216538190841675 0.3711010217666626
	Train VS Validation: 0.3668597340583801 0.3451758921146393
	Train VS Validation: 0.3152296841144562 0.356678307056427
	Train VS Validation: 0.2906443476676941 0.329077810049057
	Train VS Validation: 0.29306715726852417 0.3359682857990265
	Train VS Validation: 0.30180081725120544 0.320851594209671
	Train VS Validation: 0.3180691599845886 0.3172842562198639
	Train VS Validation: 0.32251113653182983 0.31600111722946167
	Train VS Validation: 0.2723208963871002 0.3341096341609955
	Train VS Validation

### Test

In [10]:
model_test = Model(MODEL_NAME, saved_model=MODEL_NAME+".final")

INFO:tensorflow:Restoring parameters from data/models/baseline-05.final


In [11]:
model_test.test(
    test_data, test_imgs_left, test_imgs_right
)

0.29879111

### Results


| Name | Epochs | Batch Size | Learning rate  | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| baseline-01 | 20 | 512 | 0.005 | 0.1395065039396286 | 0.19760280847549438 | 0.19909994 |
| baseline-02 | 25 | 256 | 0.001 | 0.07893882691860199 | 0.15243780612945557 | 0.15074611 |
| baseline-03 | 40 | 256 | 0.0001 | 0.057024769484996796 | 0.1646086573600769 | 0.16265412 |
| baseline-04 | 100 | 512 | 0.0001 | 0.03750447556376457 | 0.1642383337020874 | 0.16351545 |
| baseline-05 | 200 | 1024 | 0.0001 | 0.029262783005833626 | 0.1729333996772766 | 0.17122823 |